In [315]:
#Python implementation of a simulated queueing system with any number of servers
    #inspired by a homework assignment for ORIE5129: Data Science for E-retail and the sharing economy

In [316]:
import random
import math
import sys
from scipy import stats

In [317]:
# Initialization subroutines:

def initSimData(endTime = 100, meanInterarrival = 0.5, meanService = [0.8, 0.9]):
    # endTime: time when the sim ends
    # meanInterarrival: The expected time between arrivals (for sampling the exp dist)
    # meanService: A LIST of expected service times for each server
        # eg, for a two server queue with mean service times 0.8 and 0.9, meanService = [0.8, 0.9]
        # Note: order IS important bc the customer "prefers" servers with lower index (see findEmptyServer)
    
    numServers = len(meanService)
    time = 0
    inQueue = 0
    inServer = [0 for server in range(numServers)]
    return (time, inServer, inQueue, endTime, meanInterarrival, meanService, numServers)

def initMetrics():
    numArrivals = 0 #number of arrivals
    totalQueueTime = 0 # total time spent in line by arriving customers
    totalSysTime = 0 # total time spent in the system by arriving customers
    s1BusyTime = 0 # total time s1 is busy
    s1s2BusyTime = 0 # total time s1 and s2 are both busy
    return (numArrivals, totalQueueTime, totalSysTime, s1BusyTime, s1s2BusyTime)

# Functionality of the future event list:

def addEvent(eventType, eventTime): #add an arrival/departure/end event to the future event list (fel)
    global fel
    event = (eventType, eventTime)
    fel.append(event)

def deleteEvent(index): #delete an event from the future event list (after the event logic is executed)
    global fel
    fel.pop(index)
    
def findNextEvent(): #find the event in the fel with the earliest time and return its index
    global fel
    earliestTime = 1e30
    earliestIndex = -1
    for i in range(len(fel)):
        event = fel[i]
        eventTime = event[1]
        if eventTime < earliestTime:
            earliestTime = eventTime
            earliestIndex = i
    return earliestIndex

# Other subroutines:

def findEmptyServer(): #returns the smallest index i: inServer[i] = 0 (index of first empty server)
    global inServer
    for serverIndex in range(numServers):
        if inServer[serverIndex] == 0:
            return serverIndex
    print("Error: no empty servers")
    sys.exit(2)

def findDepartingServer(eventType): #returns the index of the server from which the customer departed
    return int(eventType[1:])

def getInSystem(): #returns the number of customers in the system right now
    global inQueue, inServer
    n = inQueue + sum(inServer)
    return n

# Random sampling:

def generateInterarrival(): #generate an interarrival time from the exponential distribution
    sample = random.expovariate(1/meanInterarrival)
    return sample

def generateService(serverIndex): #generate a service time from the exponential distribution
    global meanService #list of mean service times for each server
    meanServerTime = meanService[serverIndex] #finds the mean ST of the relevant server in the meanService list
    sample = random.expovariate(1/meanServerTime) #generates a sample from the exp dist with the correct mean
    return sample

# Event handling logic and metric calculation:

def handleArrival(eventTime):
    global inQueue, inServer, time, numArrivals, totalQueueTime, totalSysTime, s1BusyTime, s1s2BusyTime
    
    #increment the variables used to calculate the metrics
    dTime = eventTime - time #time since the last event
    numArrivals += 1 #increment the arrival count
    s1BusyTime += inServer[0]*dTime #if s1 is busy, add the time since the last event
    s1s2BusyTime += inServer[0]*inServer[1]*dTime #if both s1 and s2 busy, add the time since the last event
    totalSysTime += getInSystem()*dTime #add the time spent by all customers in the system since the last event
    totalQueueTime += inQueue*dTime #add the cumulative time spent by all in queue since last event
    
    #sample IA time and schedule next arrival event
    ia = generateInterarrival()
    addEvent("a", eventTime + ia)
    
    #direct the arrival to the correct server or queue
    if getInSystem() >= numServers:
        inQueue += 1
    else:
        serverIndex = findEmptyServer()
        inServer[serverIndex] = 1
        st = generateService(serverIndex)
        addEvent("d"+str(serverIndex), eventTime + st)
    time = eventTime
                
def handleDeparture(eventTime, serverIndex):
    global inQueue, inServer, time, totalQueueTime, totalSysTime, s1BusyTime, s1s2BusyTime
    
    #metrics:
    dTime = eventTime - time #time since the last event
    totalSysTime += getInSystem()*dTime #add the cumulative time spent by all customers since the last event
    totalQueueTime += inQueue*dTime #add the cumulative time spent by all in queue since last event
    s1BusyTime += inServer[0]*dTime #if s1 is busy, add the time since the last event
    s1s2BusyTime += inServer[0]*inServer[1]*dTime #if both s1 and s2 busy, add the time since the last event
    
    #If there are customers waiting, the next in line goes immediately to the open server
    if inQueue > 0:
        inQueue -= 1
        st = generateService(serverIndex)
        addEvent("d"+str(serverIndex), eventTime + st)
    
    #if the queue is empty, the relevant server is no longer busy
    else:
        inServer[serverIndex] = 0
    time = eventTime
    
def handleEnd(eventTime):
    global inQueue, inServer, time, totalQueueTime, totalSysTime, s1BusyTime, s1s2BusyTime
    
    #metrics:
    dTime = eventTime - time #time since the last event
    totalSysTime += getInSystem()*dTime #add the cumulative time spent by all customers since the last event
    totalQueueTime += inQueue*dTime #add the cumulative time spent by all in queue since last event
    s1BusyTime += inServer[0]*dTime #if s1 is busy, add the time since the last event
    s1s2BusyTime += inServer[0]*inServer[1]*dTime #if both s1 and s2 busy, add the time since the last event
    
    time = eventTime #update time. Sim loop will terminate since eventType != "e" is False

In [318]:
#Main simulation loop

replications = 100
#random.seed(1) #set ^reps = 1 if this line is un-commented

metrics = {"Arrivals" : [], 
           "Average time spent in queue" : [], 
           "Average queue length" : [], 
           "Average time spent in system" : [], 
           "Average customers in system" : [],
           "Fraction of the time server 1 is busy" : [],
           "Fraction of the time both servers are busy" : []
          }

for r in range(replications):

    (time, inServer, inQueue, endTime, meanInterarrival, meanService, numServers) = initSimData()
    (numArrivals, totalQueueTime, totalSysTime, s1BusyTime, s1s2BusyTime) = initMetrics()

    fel = list() #future event list
    ia = generateInterarrival()
    addEvent("a", ia) # schedule first arrival event
    addEvent("e", endTime) # schedule end of sim event
    eventType = "s" #initialize to dummy value

    while eventType != "e":
        earliestIndex = findNextEvent()
        earliestEvent = fel[earliestIndex]
        earliestType = earliestEvent[0]
        earliestTime = earliestEvent[1]

        if earliestType == "a":
            handleArrival(earliestTime)

        elif earliestType[0] == "d": #if there is a departure event
            server = findDepartingServer(earliestType) #find the server index of the departure event 
            handleDeparture(earliestTime, server) #execute departure logic for this server

        elif earliestType == "e":
            handleEnd(earliestTime)

        else:
            print("Invalid event type " + earliestType)
            sys.exit(1)

        deleteEvent(earliestIndex)
        eventType = earliestType

    #Compute metrics and inserting results into the metrics dictionary
    metrics["Arrivals"].append( numArrivals )
    metrics["Average time spent in queue"].append( totalQueueTime/numArrivals )
    metrics["Average queue length"].append( totalQueueTime/time )
    metrics["Average time spent in system"].append( totalSysTime/numArrivals )
    metrics["Average customers in system"].append( totalSysTime/time )  
    metrics["Fraction of the time server 1 is busy"].append( s1BusyTime/time ) 
    metrics["Fraction of the time both servers are busy"].append( s1s2BusyTime/time )

# #print the result of the first replication
# for key in metrics.keys():                      
#     print(key + ":", metrics[key][0])

In [319]:
# Estimate the expected value of each metric with a 95% CI

Z = scipy.stats.norm.ppf(.975)

metrics_avg = {}
metrics_stdev = {}
metrics_CI = {}
for key in metrics.keys():
    
    #sample mean of each metric
    metrics_avg[key] = sum(metrics[key])/replications
    
    #sample stdev of each metric
    squares = 0
    for i in range(replications):
        squares += (metrics_avg[key] - metrics[key][i])**2
    metrics_stdev[key] =  math.sqrt( squares / (replications-1) ) #ensure replications > 1
   
    #95% CI on the expected value of each metric: CI = X +- Z*stdev/sqrt(reps)
    val = Z * metrics_stdev[key] / math.sqrt(replications)
    metrics_CI[key] = ( metrics_avg[key] - val , metrics_avg[key] + val)
    
    #print the CI for each metric
    print(key + ":", metrics_CI[key])

Arrivals: (196.83843953512974, 202.28156046487027)
Average time spent in queue: (1.3948814260241185, 1.839004489338477)
Average queue length: (2.7996420610294424, 3.8113196314338804)
Average time spent in system: (2.208866093763841, 2.6610302536749972)
Average customers in system: (4.411017298062104, 5.461318563643879)
Fraction of the time server 1 is busy: (0.8300798945994403, 0.8538601970143302)
Fraction of the time both servers are busy: (0.7174338622162539, 0.75484479688977)
